In [4]:
import pandas as pd
from PIL import Image


def scale_polygon(polygon, scale=1.0):
    """Scales a polygon outward from its centroid."""
    xs, ys = zip(*polygon)
    cx, cy = sum(xs) / len(xs), sum(ys) / len(ys)
    return [(cx + scale * (x - cx), cy + scale * (y - cy)) for x, y in polygon]

def process_labels(df_filtered, label_dir):
    for idx, row in df_filtered.iterrows():
        label_lines = []

        # Process boilers (class 0)
        for poly in row['polygons_boil']:
            scaled_poly = scale_polygon(poly, scale=1.0)
            normalized_poly = [(x / image_width, y / image_height) for x, y in scaled_poly]
            poly_str = " ".join(f"{x:.6f} {y:.6f}" for x, y in normalized_poly)
            label_lines.append(f"0 {poly_str}")

        # Process panels (class 1)
        for poly in row['polygons_pan']:
            scaled_poly = scale_polygon(poly, scale=1.0)
            normalized_poly = [(x / image_width, y / image_height) for x, y in scaled_poly]
            poly_str = " ".join(f"{x:.6f} {y:.6f}" for x, y in normalized_poly)
            label_lines.append(f"1 {poly_str}")

        # Save to .txt file
        label_path = os.path.join(label_dir, f"{row['img_id']}.txt")
        with open(label_path, "w") as f:
            f.write("\n".join(label_lines))


In [5]:
import os
import pandas as pd

# Define the root directories for labels
drone_label_dir_train = r"C:\Users\gnvca\OneDrive\Desktop\JP\model04\drone\dataset\labels\train"
drone_label_dir_val = r"C:\Users\gnvca\OneDrive\Desktop\JP\model04\drone\dataset\labels\val"
satellite_label_dir_train = r"C:\Users\gnvca\OneDrive\Desktop\JP\model04\satellite\dataset\labels\train"
satellite_label_dir_val = r"C:\Users\gnvca\OneDrive\Desktop\JP\model04\satellite\dataset\labels\val"

# Create directories if they do not exist
os.makedirs(drone_label_dir_train, exist_ok=True)
os.makedirs(drone_label_dir_val, exist_ok=True)
os.makedirs(satellite_label_dir_train, exist_ok=True)
os.makedirs(satellite_label_dir_val, exist_ok=True)

image_width, image_height = 512, 512  # Replace with actual size if dynamic

# Load the pickled data files (train and validation)
train_df = pd.read_pickle('Model_Train.pkl')
val_df = pd.read_pickle('Model_Val.pkl')


# Process training and validation datasets for drone and satellite
for img_origin in ['D', 'S']:
    # Filter the dataframe based on the image origin
    train_filtered = train_df[train_df['img_origin'] == img_origin]
    val_filtered = val_df[val_df['img_origin'] == img_origin]

    # Set the label directories based on the image origin and train/val
    if img_origin == 'D':
        train_label_dir = drone_label_dir_train
        val_label_dir = drone_label_dir_val
    else:
        train_label_dir = satellite_label_dir_train
        val_label_dir = satellite_label_dir_val

    # Process and save training labels
    process_labels(train_filtered, train_label_dir)

    # Process and save validation labels
    process_labels(val_filtered, val_label_dir)


In [8]:
from ultralytics import YOLO
import yaml

# Load your pre-trained YOLOv8 model
model = YOLO(r"C:\Users\gnvca\OneDrive\Desktop\JP\yolo11m-seg.pt")

# ✅ Load hyperparameters manually from hyp.yaml
with open(r"C:\Users\gnvca\OneDrive\Desktop\JP\model04\drone\hyp.yaml", "r") as f:
    hyp_params = yaml.safe_load(f)  # Converts YAML to dictionary

# ✅ Pass hyperparameters as individual keyword arguments
model.train(
    data=r"C:\Users\gnvca\OneDrive\Desktop\JP\model04\drone\yolo.yaml",  # Dataset config
    epochs=75,      # Total training epochs
    imgsz=512,       # Image size (try 768, 1024 for better accuracy)
    batch=16,        # Batch size (increase if GPU allows)
    project = 'ModelBuild',
    name='model04_yoloseg_drone_2',                 # Name of the specific experiment folder (creates a new folder)
    exist_ok=True,    
    amp=True,        # Automatic Mixed Precision (for speedup)
    conf=0.2,        # Confidence threshold for predictions
    iou=0.6,         # Intersection Over Union threshold
    optimizer="AdamW", # Optimizer choice
    device=0,        # Use GPU 0
    workers=4,       # Number of data workers (depends on CPU)
    cache=True,      # Cache images to RAM for faster training
    val=True,        # Run validation during training
    verbose=True,    # Print logs during training
    save=True,       # Save best and final models
    **hyp_params  # ✅ This applies all values from hyp.yaml
)


Ultralytics 8.3.94  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=segment, mode=train, model=C:\Users\gnvca\OneDrive\Desktop\JP\yolo11m-seg.pt, data=C:\Users\gnvca\OneDrive\Desktop\JP\model04\drone\yolo.yaml, epochs=75, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=True, device=0, workers=4, project=ModelBuild, name=model04_yoloseg_drone, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.2, iou=0.6, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames

train: Scanning C:\Users\gnvca\OneDrive\Desktop\JP\model04\drone\dataset\labels\train... 1998 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1998/1998 [00:02<00:00, 772.98it/s]

train: WARNING  C:\Users\gnvca\OneDrive\Desktop\JP\model04\drone\dataset\images\train\IDcxTl9zDOWTNH.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\gnvca\OneDrive\Desktop\JP\model04\drone\dataset\images\train\IDpNecgm1Opkw.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.2764156 2.093781 ]


train: New cache created: C:\Users\gnvca\OneDrive\Desktop\JP\model04\drone\dataset\labels\train.cache
WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (1.1GB RAM): 100%|██████████| 1997/1997 [00:17<00:00, 113.20it/s]
val: Scanning C:\Users\gnvca\OneDrive\Desktop\JP\model04\drone\dataset\labels\val... 490 images, 0 backgrounds, 0 corrupt: 100%|██████████| 490/490 [00:00<00:00, 683.46it/s]

...

                   all        490        935      0.852      0.446      0.592       0.44      0.846      0.436      0.583       0.39



75 epochs completed in 1.819 hours.
Optimizer stripped from ModelBuild\model04_yoloseg_drone\weights\last.pt, 45.1MB
Optimizer stripped from ModelBuild\model04_yoloseg_drone\weights\best.pt, 45.1MB

Validating ModelBuild\model04_yoloseg_drone\weights\best.pt...
Ultralytics 8.3.94  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
YOLO11m-seg summary (fused): 138 layers, 22,336,854 parameters, 0 gradients, 123.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:08<00:00,  1.81it/s]


                   all        490        935      0.819       0.46      0.675      0.505       0.81      0.449      0.666      0.427
                boiler         10         21          1      0.143      0.571      0.435          1      0.143      0.571      0.338
                 panel        487        914      0.638      0.778      0.779      0.575       0.62      0.756       0.76      0.516
Speed: 0.1ms preprocess, 14.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to ModelBuild\model04_yoloseg_drone


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021938D50F40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0

In [9]:
from ultralytics import YOLO
import yaml

# Load your drone-trained model weights
model = YOLO(r"C:\Users\gnvca\OneDrive\Desktop\JP\ModelBuild\model04_yoloseg_drone\weights\last.pt")

In [12]:
# Optional: Load hyperparameters if you want the same tuning
with open(r"C:\Users\gnvca\OneDrive\Desktop\JP\model04\satellite\hyp.yaml", "r") as f:
    hyp_params = yaml.safe_load(f)

# Train on satellite images
model.train(
    data=r"C:\Users\gnvca\OneDrive\Desktop\JP\model04\satellite\yolo.yaml",
    epochs=75,
    imgsz=512,
    batch=32,
    project='ModelBuild',
    name='model04_yoloseg_satellite',
    exist_ok=True,
    amp=True,
    conf=0.2,
    iou=0.6,
    optimizer="AdamW",
    device=0,
    workers=4,
    cache=True,
    val=True,
    verbose=True,
    save=True,
    freeze = [0, 1, 2],
    **hyp_params  # Optional: only if you're customizing learning behavior
)

Ultralytics 8.3.94  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=segment, mode=train, model=C:\Users\gnvca\OneDrive\Desktop\JP\ModelBuild\model04_yoloseg_drone\weights\last.pt, data=C:\Users\gnvca\OneDrive\Desktop\JP\model04\satellite\yolo.yaml, epochs=75, time=None, patience=100, batch=32, imgsz=512, save=True, save_period=-1, cache=True, device=0, workers=4, project=ModelBuild, name=model04_yoloseg_satellite, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=[0, 1, 2], multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.2, iou=0.6, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

train: Scanning C:\Users\gnvca\OneDrive\Desktop\JP\model04\satellite\dataset\labels\train.cache... 441 images, 0 backgrounds, 0 corrupt: 100%|██████████| 441/441 [00:00<?, ?it/s]

WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (0.3GB RAM): 100%|██████████| 441/441 [00:00<00:00, 3266.70it/s]
val: Scanning C:\Users\gnvca\OneDrive\Desktop\JP\model04\satellite\dataset\labels\val.cache... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]

WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

      33/75      9.82G      1.784      2.696      3.439       1.51         93        512: 100%|██████████| 14/14 [00:52<00:00,  3.74s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:08<00:00,  4.33s/it]

                   all        120        218       0.53      0.443      0.457      0.233      0.481      0.395      0.419      0.192



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      34/75      9.81G      1.775      2.675      3.447      1.507        113        512:  71%|███████▏  | 10/14 [00:47<00:19,  4.76s/it]


KeyboardInterrupt: 